In [14]:
# import standard libraries
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [15]:
# import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [16]:
# Add OOM errrors by setting GPU Memory Consumption Growth
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#   tf.config.experimental.set_memory_growth(gpu, True)

MOVE DATASET FROM LFW TO DATA FOLDER

In [17]:
# Set up paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [18]:
# Make directories
os.makedirs(POS_PATH, exist_ok=True)
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANC_PATH, exist_ok=True)

In [19]:
# for directory in os.listdir('lfw'): # loop directory
#   for file in os.listdir(os.path.join('lfw', directory)): # loop files
#     EX_PATH = os.path.join('lfw', directory, file)
#     NEW_PATH = os.path.join(NEG_PATH, file)
#     os.replace(EX_PATH, NEW_PATH)

COLLECT POSITIVE AND ANCHOR CLASSES

In [20]:
import uuid

In [21]:
# uuid?? -> "??" Used for library information tracking 

In [22]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
  ret, frame  = cap.read()
  frame = frame[120: 120 + 250, 200:200 + 250, :]
  cv2.imshow('Image Collection', frame)

  # Collect Anchor Images
  if cv2.waitKey(1) & 0xFF == ord('a'):
    # Create unique file path
    imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
    # Write out anchor image
    cv2.imwrite(imgname, frame)

  # Collect Positive Images
  if cv2.waitKey(1) & 0xFF == ord('p'):
    # Create unique file path
    imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
    # Write out anchor image
    cv2.imwrite(imgname, frame)

  # Breaking gracefully
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()